In [6]:
#Notebook for Crime and Weather Data Wrangling
#ASSUMPTIONS
#We are only using 2016 date
#For weather we consider below fields:
  #Temperature
  #Pressure
  #Humidity
  #Wind
  #Precipitation
  #Clouds
#We are only focusing on top crime


#SEPARATE for crime count vs weather Carlos, Doris
#1. Agregate data based on crime count -> Larceny-from vehicle
#2. Agregate by date
#3. Get weather data per date
#4. Test correlations

#SEPARATE for crime count vs ice-cream shop Rabia, Isaiah 
#1. Agregate data based on crime count -> Larceny-from vehicle
#2. Agregate by date to get crime count ->5
#2. Add number of shops per coordinate per date -> take max count
#3. Correlate crime count vs number of shops

#VISUALIZATIONS & HYPOTHESIS TESTING FOR CRIME vs WEATHER
#location map of of count of crimes in Atlanta (put in the first part)
#pie chart of all types of crimes
#bar plot of LARCENY-FROM VEHICLE	 count per month in 2016
#ditplot of crime count
#distplot of max temps & each weather variable (pressure,humidity, wind, precipitation, cloud)
#scatter plot of crime count per date vs temperature
#scatter plot of crime count per date vs each weather variable (pressure,humidity, wind, precipitation, cloud)

#VISUALIZATIONS & HYPOTHESIS TESTING FOR CRIME vs ICE-CREAM
#distplot of ice-crem shop count
#scatter plot of crime count per date vs ice-cream shop count
#gplot of crime with max temp and ice-cream shop vcicnity (pick one row)

#CONCLUSION & OBSERVATIONS
#Result of testing hypothesis
#fun facts!

In [7]:
#Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import os
%matplotlib inline

In [8]:
#Load Atlanta Crime Dataset taken from: https://data.world/bryantahb/crime-in-atlanta-2009-2017
path = os.path.join('..','Resources','atlcrime.csv')
df = pd.read_csv(path)
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head()

/Users/carlospazos/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,crime,number,date,location,beat,neighborhood,npu,lat,long
0,LARCENY-NON VEHICLE,103040029,10/31/2010,610 SPRING ST NW,509,Downtown,M,33.77101,-84.38895
1,AUTO THEFT,103040061,10/31/2010,850 OAK ST SW,401,West End,T,33.74057,-84.41680
2,LARCENY-FROM VEHICLE,103040169,10/31/2010,1344 METROPOLITAN PKWY SW,301,Capitol View Manor,X,33.71803,-84.40774
3,AUTO THEFT,103040174,10/31/2010,1752 PRYOR RD SW,307,Betmar LaVilla,Y,33.70731,-84.39674
4,LARCENY-NON VEHICLE,103040301,10/31/2010,JOHN WESLEY DOBBS AVE NE / CORLEY ST NE,604,Old Fourth Ward,M,33.75947,-84.36626


In [11]:
#Explore the Dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270688 entries, 0 to 270687
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   crime         270688 non-null  object 
 1   number        270688 non-null  int64  
 2   date          270688 non-null  object 
 3   location      270686 non-null  object 
 4   beat          270688 non-null  object 
 5   neighborhood  258928 non-null  object 
 6   npu           268592 non-null  object 
 7   lat           270688 non-null  float64
 8   long          270688 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 18.6+ MB


In [12]:
df.describe()

,number,lat,long
count,2.706880e+05,270688.000000,270688.000000
mean,4.705194e+08,33.755246,-84.408178
std,6.768886e+09,0.045540,0.047298
min,7.269234e+07,33.460100,-84.550500
25%,1.029113e+08,33.727780,-84.433880
50%,1.227606e+08,33.755380,-84.397520
75%,1.432412e+08,33.780140,-84.374090
max,1.704427e+11,33.886130,-84.286410


In [26]:
#Split Date column into day, month, and year to be able to call OpenWeather Statistics API
dsplit = df['date'].apply(lambda x: x.split("/"))
months = []
days = []
years = []

for item in dsplit:
    months.append(item[0])
    days.append(item[1])
    years.append(item[2])

#Create Month, Day and Year columns
df['Month'] = months
df['Day'] = days
df['Year'] = years

df.head()

,crime,number,date,location,beat,neighborhood,npu,lat,long,Month,Day,Year
0,LARCENY-NON VEHICLE,103040029,10/31/2010,610 SPRING ST NW,509,Downtown,M,33.77101,-84.38895,10,31,2010
1,AUTO THEFT,103040061,10/31/2010,850 OAK ST SW,401,West End,T,33.74057,-84.41680,10,31,2010
2,LARCENY-FROM VEHICLE,103040169,10/31/2010,1344 METROPOLITAN PKWY SW,301,Capitol View Manor,X,33.71803,-84.40774,10,31,2010
3,AUTO THEFT,103040174,10/31/2010,1752 PRYOR RD SW,307,Betmar LaVilla,Y,33.70731,-84.39674,10,31,2010
4,LARCENY-NON VEHICLE,103040301,10/31/2010,JOHN WESLEY DOBBS AVE NE / CORLEY ST NE,604,Old Fourth Ward,M,33.75947,-84.36626,10,31,2010


In [30]:
#We will only focus on crime happenning in year 2016
crime = df.loc[df['Year']=='2016']
crime.reset_index(drop =True, inplace=True)
crime.head()

,crime,number,date,location,beat,neighborhood,npu,lat,long,Month,Day,Year
0,HOMICIDE,163051391092,10/31/2016,1205 METROPOLITAN AVE SE,612,East Atlanta,W,33.74173,-84.34749,10,31,2016
1,AUTO THEFT,163030548,10/31/2016,180 PONCE DE LEON AVE NE,505,Midtown,E,33.77285,-84.38149,10,31,2016
2,LARCENY-FROM VEHICLE,163041611,10/31/2016,1783 JOHNSON RD NW,110,Rockdale,G,33.78547,-84.44593,10,31,2016
3,LARCENY-FROM VEHICLE,163050183,10/31/2016,1000 NORTHSIDE DR NW,501,Home Park,E,33.78196,-84.40835,10,31,2016
4,AUTO THEFT,163050218,10/31/2016,2264 COUNTRY CLUB DR SW,409,Southwest,R,33.69384,-84.49398,10,31,2016


In [31]:
crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29131 entries, 0 to 29130
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   crime         29131 non-null  object 
 1   number        29131 non-null  int64  
 2   date          29131 non-null  object 
 3   location      29131 non-null  object 
 4   beat          29131 non-null  object 
 5   neighborhood  27795 non-null  object 
 6   npu           28877 non-null  object 
 7   lat           29131 non-null  float64
 8   long          29131 non-null  float64
 9   Month         29131 non-null  object 
 10  Day           29131 non-null  object 
 11  Year          29131 non-null  object 
dtypes: float64(2), int64(1), object(9)
memory usage: 2.7+ MB
